In [68]:
import json
with open('svelte-discord.json') as f:
    data = json.load(f)

In [148]:
from operator import attrgetter

channel_ids = {k: v['name'] for k, v in data['meta']['channels'].items()}
user_ids = {k: v['name'] for k, v in data['meta']['users'].items() }

usernames = data['meta']['users']
users = { i: {'user_name': usernames[uid]['name'], 'user_id': uid}
         for i, uid in enumerate(data['meta']['userindex']) 
        }
uids = { data['user_id']: data['user_name']
        for i, data in users.items()
}

channel_data = {
    channel_ids[channel_id]: channel_data
    for channel_id, channel_data in data['data'].items()
}

channels = {}
for name, d in channel_data.items():
    channels.update({name: []})
    for message_id, message in d.items():
        message.update({'message_id': message_id})
        userinfo = {
            'user_name': users[message['u']]['user_name'],
            'user_id': users[message['u']]['user_id']
        }
        message.update(userinfo)
        channels[name].append(message)
        
channels['future'][0]
uids

{'202944491408195584': 'Kazzkiq',
 '497813112586633226': 'pngwn',
 '310138603521179648': 'Tholle',
 '401880574454923264': 'cristianl',
 '253428717575995402': 'lukeed',
 '501779242057728010': 'mindrones',
 '575012643363094539': 'rzvdaniel',
 '234040585130999810': 'Gumug',
 '265714579785515025': 'Glitch',
 '399349371046592513': 'OrdinaryJellyfish',
 '177239516833251331': 'skullclown',
 '107176742342402048': 'silentworks',
 '96737940138377216': 'Tivac',
 '103063566071922688': 'frosT',
 '568779478352920598': 'nick walton',
 '163055914734190592': 'Conduitry',
 '104950858734694400': 'TehShrike',
 '258258235159609346': 'natec425',
 '451242710486220800': 'stace',
 '335806193379442691': 'rich_harris',
 '143376740113317888': 'b-fuze (Mike32)',
 '570979556853284885': 'mmilo1',
 '435292558424276994': 'H with a K',
 '391924106355933187': 'Lars Klopstra',
 '417055500585140224': 'sander',
 '106758707399921664': 'Morphyish',
 '105806481084960768': 'MikeMatrix',
 '151557606991724544': 'fractal',
 '1869

In [189]:
def write_json(data, fname):
    with open(fname, 'w') as f_out:
        f_out.write(json.dumps(channels, indent=4))


headers = ['channel', 'user_name', 'm', 't', 'u', 'user_id', 'message_id']
tabular = [
    # channel, from, 
    [c, m['user_name'], m['m'], m['t'], m['u'], m['user_id'], m['message_id']]
    for c, messages in channels.items()
    for m in messages
]


pattern = re.compile(r'(<@)(\d+)(>)')

def get_mention(uid) -> str:
    try:
        mention = uids[uid]
    except:
        mention = f'<@{uid}>'
    return mention

def get_mentions(message, pattern=pattern):
    try:
        results = re.findall(pattern, message)
        return [get_mention(r[1]) for r in results]
    except:
        return ''

try:
    headers.append('mentions')
    for t in tabular:
        message = t[2]
        t.append(get_mentions(message))
except:
    # only add mentions if it actually works
    pass

tabular[0:3]

[['general',
  'arxpoetica',
  'Discuss Svelte',
  1529245407783,
  30,
  '301383866340605953',
  '457913171844399104',
  []],
 ['general',
  'rich_harris',
  '',
  1529245514271,
  19,
  '335806193379442691',
  '457913618487312386',
  []],
 ['general',
  'arxpoetica',
  "<@335806193379442691> I just passed the buck. 😂 You're now the admin.",
  1529245826990,
  30,
  '301383866340605953',
  '457914930126127105',
  ['rich_harris']]]

In [195]:
import collections
import nltk
import numpy
import pandas
import re


df = pandas.DataFrame(tabular, columns=headers)
# todo: replace tags with @username
# x = s[2]
# re.match(pat, x).group(2)

def replace_username(mat) -> str:
    try:
        uid = mat.group(2)
        return f'@{uids[uid]}'
    except:
        return f'<@{mat.group(2)}>'

df['m'] = df['m'].str.replace(pattern, replace_username, regex=True)
df


# pat

,channel,user_name,m,t,u,user_id,message_id,mentions
0,general,arxpoetica,Discuss Svelte,1529245407783,30,301383866340605953,457913171844399104,[]
1,general,rich_harris,,1529245514271,19,335806193379442691,457913618487312386,[]
2,general,arxpoetica,@rich_harris I just passed the buck. 😂 You're ...,1529245826990,30,301383866340605953,457914930126127105,[rich_harris]
3,general,arxpoetica,no touchbacks,1529245838200,30,301383866340605953,457914977144143885,[]
4,general,arxpoetica,I think this is worth a trial / shot,1529245866762,30,301383866340605953,457915096941723652,[]
5,general,arxpoetica,The nice thing about it is we'll be able to th...,1529245896439,30,301383866340605953,457915221416214538,[]
6,general,arxpoetica,Oops. I think I unwittingly did something by m...,1529246085680,30,301383866340605953,457916015150366723,[]
7,general,arxpoetica,I think we should create a separate #sapper ch...,1529246122209,30,301383866340605953,457916168364228611,[]
8,general,arxpoetica,Some basics: https://support.discordapp.com/hc...,1529246216469,30,301383866340605953,457916563719454720,[]
9,general,Conduitry,,1529246218743,15,163055914734190592,457916573257302019,[]
